In [1]:
import openai
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
load_dotenv()

True

In [3]:
openai_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

In [4]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

In [5]:
DATBASE  = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [69]:
retriever=DATBASE.as_retriever(search_kwargs={"k": 2, "score_threshold": 0.5})

In [7]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [8]:
HISTORY = []
    
HISTORY.append(HumanMessage(content="Вася"))
HISTORY.append(AIMessage(content="Это Вася"))
HISTORY.append(HumanMessage(content="Петя"))
HISTORY.append(AIMessage(content="Это Петя"))

In [9]:
HISTORY

[HumanMessage(content='Вася'),
 AIMessage(content='Это Вася'),
 HumanMessage(content='Петя'),
 AIMessage(content='Это Петя')]

In [66]:


prompt_template = ChatPromptTemplate.from_messages(
    [   
        SystemMessagePromptTemplate.from_template("Ты полезный асистент, дай ответ исходя из контекста и диалога: {context}"),
        *HISTORY,
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

In [67]:
documents = RunnableParallel(
    {"context": retriever, "user_input": RunnablePassthrough()}
)

In [68]:
documents.invoke("Кто ты?")

{'context': [], 'user_input': 'Кто ты?'}

In [70]:
chain = documents | prompt_template | model

In [71]:
res = await chain.ainvoke("Как тебя зовут")  

In [102]:
res

AIMessage(content='Вася, Петя', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 504, 'total_tokens': 511}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-65cb873b-4125-4300-b878-47a793857e62-0')